# NPC Position MS 
# New data from chelle - nov 23, 2019
## Testing hypotheses: 
### position influences charactheristics (SST and velocity) of the California and Alaska Currents
### this varies by season
### in winter, bifurcation (alternating cal and alaska current velocity) is dominating, while SST covaries. 
***

# Data 
### Seasonal averages in the NPC, Bifurcation, California Current and Alaska Current boxes: SST and velocity anomalies
### Position of the NPC bifurcation at 165 and 135 - seasonal average
### PDO and NPGO seasonal averages
#### Seasons are defined DJF, MAM, JAS, OND
#### Data from 2000-2018 ( avoiding 1997-1999)
### Sources: SST (), current velocity (OSCAR), position (AVISO?), PDO ?, NPGO?
***

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as stat
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import warnings
warnings.simplefilter('ignore') # filter some warning messages

In [ ]:
## seasonal data already averaged by in boxes. anomalies - including position
seasanom = xr.open_dataset('../data/bifdatams_seas_long_26nov2019.nc')
seasanom.close()
seasanom

In [ ]:
newseas_tmp=seasanom.to_dataframe()
newseas_tmp['month'] = pd.to_datetime(newseas_tmp.index.values).month.values
newseas_tmp['Season']=1
newseas_tmp['Season'][newseas_tmp['month']==5]=2
newseas_tmp['Season'][newseas_tmp['month']==8]=3
newseas_tmp['Season'][newseas_tmp['month']==11]=4
iy=2000
newseas = newseas_tmp.loc['2000-01-01':'2018-12-31']
newseas.head()

In [ ]:
plt.figure()
plt.plot(newseas['NPC165'],label='NPC165')
plt.plot(newseas['NPC135'],label='NPC135')
plt.grid(True)
plt.legend()
plt.show()

plt.figure()
plt.plot(newseas['V_CAC'],label='V_CAC')
plt.plot(newseas['V_CAK'],label='V_CAK')
plt.grid(True)
plt.legend()
plt.show()

plt.figure()
plt.plot(newseas['SST_CAC'],label='SST_CAC')
plt.plot(newseas['SST_CAK'],label='SST_CAK')
plt.plot(newseas['SST_NPC'],label='SST_NPC')
plt.plot(newseas['SST_Bif'],label='SST_Bif')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
## calculate PCA per season
## V currents Cal and Alaska and Positions
PC1v4=np.zeros((int(len(newseas)/4),4))
PC2v4=np.zeros((int(len(newseas)/4),4))

for season in range(1,5):
## make currents table
    currs=newseas[newseas['Season']==season][['V_CAK','V_CAC','NPC165','NPC135']].copy()
               
    # standardize the data set
    for i in list(currs):
        currs[i]=scale(currs[i])
    #currs.head()

    # covariance matrix
    cov_mx_curr = PCA(n_components = 2)
    cov_mx_curr.fit(currs)
    eigvalues = cov_mx_curr.explained_variance_
    variance = np.round(cov_mx_curr.explained_variance_ratio_, decimals=3)*100 #calculate variance ratios
    var=np.cumsum(np.round(cov_mx_curr.explained_variance_ratio_, decimals=3)*100)
    print('\n\nSeason: '+str(season))
    print('Eigenvalues:',eigvalues)
    print('Explained Variance:', variance)
    print('Cumulative Explained Variance:',var)

    pcdfv=pd.DataFrame(cov_mx_curr.components_.T,columns=['PC1','PC2'])
    for iv,i in enumerate(list(currs)):
        pcdfv=pcdfv.rename(index={iv:i})
    print(pcdfv)
    
    PC1v4[:,season-1]=(currs.values*cov_mx_curr.components_[0,:].T).sum(axis=1)
    PC2v4[:,season-1]=(currs.values*cov_mx_curr.components_[1,:].T).sum(axis=1)
    
#PC1v4[:,0] = -1*PC1v4[:,0]
#PC2v4[:,0] = -1*PC2v4[:,0]
# adjust signs for V to be positive if both negative
PC1v4[:,2] = -1*PC1v4[:,2] # PC1 summer

In [ ]:
## ## correlate PCs with SST at NPCv 
print('Correlation PC_Vs with NPC SST')
for season in range(1,5):
    print('season: '+str(season))
    rho,ps=stat.spearmanr(PC1v4[:,season-1],newseas[newseas['Season']==season]['SST_NPC'])
    print('PC1. rho=',rho,ps)
    rho,ps=stat.spearmanr(PC2v4[:,season-1],newseas[newseas['Season']==season]['SST_NPC'])
    print('PC2. rho=',rho,ps)

print('\n\nCorrelation PC_Vs with Bif SST')
for season in range(1,5):
    print('season: '+str(season))
    rho,ps=stat.spearmanr(PC1v4[:,season-1],newseas[newseas['Season']==season]['SST_Bif'])
    print('PC1. rho=',rho,ps)
    rho,ps=stat.spearmanr(PC2v4[:,season-1],newseas[newseas['Season']==season]['SST_Bif'])
    print('PC2. rho=',rho,ps)

In [ ]:
# correlations with PDO and NPGO
pdo2 = pd.read_csv('../data/pdo.csv')
pdo2 = pdo2.rename(columns={'Unnamed: 0':'Year'})
pdo = pdo2[(pdo2['Year']>=iy)&(pdo2['Year']<=2018)]
pdo['PDO3M'] = pdo['PDO'].rolling(3, center=True).mean()

npgo2 = pd.read_csv('../data/npgo.csv')
npgo2 = npgo2.rename(columns={'Unnamed: 0':'Year'})
npgo = npgo2[(npgo2['Year']>=iy)&(npgo2['Year']<=2018)]
npgo['NPGO3M'] = npgo['NPGO'].rolling(3, center=True).mean()

mose=[0,2,5,8,11]
print('\n\nCorrelation PC_Vs with PDO')
for season in range(1,5):
    print('\nseason: '+str(season))
    pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
    rho,ps=stat.spearmanr(PC1v4[:,season-1],pdos)
    print('PC1. rho=',rho,ps)
    rho,ps=stat.spearmanr(PC2v4[:,season-1],pdos)
    print('PC2. rho=',rho,ps)

print('\n\nCorrelation PC_Vs with NPGO')
for season in range(1,5):
    print('\nseason: '+str(season))
    npgos=npgo[npgo['Month']==mose[season]]['NPGO3M']
    rho,ps=stat.spearmanr(PC1v4[:,season-1],npgos)
    print('PC1. rho=',rho,ps)
    rho,ps=stat.spearmanr(PC2v4[:,season-1],npgos)
    print('PC2. rho=',rho,ps)
    

    

In [ ]:
tempo=['Winter','Spring','Summer','Fall']
plt.figure(figsize=(8,10),dpi=200)

# winter
season = 1
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_NPC','SST_Bif']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_NPC'].values,'.-',label='-SST 165W')
plt.plot(pdos.values,'.-',label='PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

# spring
season = 2
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_NPC','SST_Bif']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_Bif'].values,'.-',label='-SST 135W')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

# summer
season = 3
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_NPC','SST_Bif']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_Bif'].values,'.-',label='-SST 135W')
plt.plot(-pdos.values,'.-',label='-PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

#fall
season = 4
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_NPC','SST_Bif']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_NPC'].values,'.-',label='-SST 165W')
plt.plot(pdos.values,'.-',label='PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

plt.tight_layout()
plt.savefig('../figures/PCtimeseries_forMS.png')
plt.show()

In [ ]:
tempo=['Winter','Spring','Summer','Fall']
plt.figure(figsize=(8,10),dpi=200)

# winter
season = 1
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_CAC','SST_CAK']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(currs['SST_CAK'].values,'.-',label='SST AC')
plt.plot(currs['SST_CAC'].values,'.-',label='SST CC')
plt.plot(pdos.values,'.-',label='PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

# spring
season = 2
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_CAC','SST_CAK']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_CAK'].values,'.-',label='-SST AC')
plt.plot(-currs['SST_CAC'].values,'.-',label='-SST CC')
plt.plot(-pdos.values,'.-',label='-PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

# summer
season = 3
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_CAC','SST_CAK']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(-currs['SST_CAK'].values,'.-',label='-SST AC')
plt.plot(-currs['SST_CAC'].values,'.-',label='-SST CC')
plt.plot(-pdos.values,'.-',label='-PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

#fall
season = 4
plt.subplot(4,1,season)
pdos=pdo[pdo['Month']==mose[season]]['PDO3M']
currs=newseas[newseas['Season']==season][['SST_CAC','SST_CAK']].copy()
plt.plot(PC1v4[:,season-1],'.-',label='PCv1')
plt.plot(currs['SST_CAK'].values,'.-',label='SST AC')
plt.plot(currs['SST_CAC'].values,'.-',label='SST CC')
plt.plot(pdos.values,'.-',label='PDO')
plt.title(tempo[season-1])
plt.xticks(range(0,19,2))
locs, labels = plt.xticks()
plt.xticks(locs,locs+iy)
plt.ylabel('SST (oC) \n Index')
plt.grid(True)
plt.legend(fontsize='x-small') 

plt.tight_layout()
plt.savefig('../figures/PCtimeseriesSST_forMS.png')
plt.show()